# Classificação de Tumores Cerebrais com YOLOv11 e Faster R-CNN

**Autores:** Beatriz Correia Santos, Gabriel Silva da Rocha

**Curso:** Mestrado Profissional em Engenharia Elétrica – UEA  

**Data:** Novembro/2025


# **INTRODUÇÃO**
Na prática clínica neuro-oncológica, a ressonância magnética (RM) é a
modalidade de imagem padrão-ouro para a detecção, caracterização e acompanhamento de tumores cerebrais. No entanto, a interpretação desses exames é uma tarefa complexa, tempo-intensiva e inerentemente dependente da experiência do especialista, estando suscetível à variabilidade inter-observador. A demanda crescente por exames de imagem, aliada à necessidade de diagnósticos rápidos e precisos, expõe a limitação do fluxo de trabalho manual, onde a fadiga pode levar a falsos-negativos ou erros de classificação da lesão. Um diagnóstico incorreto ou tardio impacta diretamente o prognóstico do paciente e a definição do plano terapêutico adequado.

Este estudo visa mitigar tais desafios através do desenvolvimento de um sistema automatizado para a classificação de imagens de RM cerebral em quatro principais categorias: glioma, meningioma, pituitário e "sem tumor". O uso da aplicação de Deep Learning (DL) fundamenta-se na capacidade comprovada das Redes Neurais Convolucionais (CNNs) em atuar como extratores robustos de características, aprendendo hierarquias complexas de padrões visuais diretamente dos dados. Modelos de DL podem identificar texturas e morfologias sutis, muitas vezes imperceptíveis ao olho humano, oferecendo um potencial significativo para aumentar a acurácia, a velocidade e a objetividade do diagnóstico, servindo como uma ferramenta poderosa de suporte à decisão clínica.

# **TRABALHOS RELACIONADOS**
A detecção e classificação de tumores cerebrais a partir de imagens de ressonância magnética têm sido amplamente estudadas na literatura recente. As abordagens tradicionais de Computer Vision vêm sendo substituídas por modelos de Deep Learning, que apresentam melhor capacidade de generalização e extração de padrões complexos.

Diversos estudos recentes utilizam o mesmo conjunto de dados empregado neste trabalho — o dataset de ressonâncias magnéticas cerebrais com quatro classes (Glioma, Meningioma, Pituitária e Sem Tumor) — evidenciando sua ampla adoção na pesquisa científica. Trabalhos como os de [Noori et al.](https://ieeexplore.ieee.org/abstract/document/10836556) (2024) e [Anwar et al.](https://ieeexplore.ieee.org/abstract/document/10890954) (2025) aplicaram técnicas de transfer learning em modelos ResNet50V2, VGG16, InceptionV3 e DenseNet121, alcançando acurácias entre 94% e 95% e F1-scores superiores a 0,93, confirmando a viabilidade desse dataset para tarefas de classificação e sua representatividade nas investigações da área médica.

De forma semelhante, [Krolik et al.](https://arxiv.org/html/2510.10250v1) (2025) exploraram abordagens híbridas que combinam classification, segmentation e object detection utilizando arquiteturas como U-Net e EfficientDet, reforçando o potencial de modelos de detecção aplicados à identificação de tumores cerebrais. O presente trabalho segue essa mesma linha, adotando as arquiteturas YOLOv11 e Faster R-CNN para realizar a detecção e posterior classificação de tumores, baseando-se também no estudo prático de [Nuruzzaman](https://www.kaggle.com/code/nuruzzamannuru/brain-tumor-yolov11-and-custom-cnn) (2025), que propôs um pipeline semelhante com YOLO e CNN personalizados para o mesmo tipo de dataset.

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
masoudnickparvar_brain_tumor_mri_dataset_path = kagglehub.dataset_download('masoudnickparvar/brain-tumor-mri-dataset')
ahmedsorour1_mri_for_brain_tumor_with_bounding_boxes_path = kagglehub.dataset_download('ahmedsorour1/mri-for-brain-tumor-with-bounding-boxes')
gabrielrocha2_duplicatas_path = kagglehub.dataset_download('gabrielrocha2/duplicatas')

print('Data source import complete.')

# **METODOLOGIA**

O dataset escolhido foi o [Brain Tumor MRI Dataset](https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset), por Masoud Nickparvar que é composto por três fontes distintas de imagens de ressonância magnética. Cada uma das três fontes trata as imagens de formas diferentes, sendo uma das fontes contendo apenas imagens com os três tipos de tumores rotulados, uma outra fonte apenas com imagens rotuladas como “com tumor” e “sem tumor”, e a outra fonte contendo imagens rotuladas com os três tipos de tumor e sem tumor também.
O conjunto de dados contém pouco mais de sete mil imagens divididas em quatro classes, nas quais estão separadas em diretórios para treino e testes de modelos. As classes são: Glioma ( ~23%), Sem Tumor ( ~29%), Pituitary ( ~24%), Meningioma  ( ~23%). Com isso, as classes estão com um pequeno desbalanceamento. As imagens estão divididas em aproximadamente 81% para treino e 19% para testes.

Para a aplicação do modelo YOLO é necessário ter todas as imagens com suas respectivas identificações de ocorrências através de caixas (bounding boxes) para que o treinamento do modelo seja feito. Para isso, foi utilizado o dataset [MRI for Brain Tumor with Bounding Boxes](https://www.kaggle.com/datasets/ahmedsorour1/mri-for-brain-tumor-with-bounding-boxes), que foi desenvolvido baseado no primeiro dataset apresentado, porém com suas devidas *labels* em formato YOLO.


In [ ]:

import matplotlib.pyplot as plt

# Function for importing data
def get_data_labels(directory, shuffle=True, random_state=0):
    """
    Function used for going into the main training directory
    whose directory has sub-class-types.
    """
    from sklearn.utils import shuffle
    import os

    # Lists to store data and labels
    data_path = []
    data_labels = []

    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)

        # Avoid MacOS storing path
        if not os.path.isdir(label_dir):
            continue

        # Going into each folder and getting image path
        for image in os.listdir(label_dir):
            image_path = os.path.join(label_dir, image)
            data_path.append(image_path)
            data_labels.append(label)

    if shuffle:
        data_path, data_labels = shuffle(data_path, data_labels, random_state=random_state)

    return data_path, data_labels

# **DISTRIBUIÇÃO DO DATASET ORIGINAL**

In [ ]:
# Setting up file paths for training and testing
USER_PATH = masoudnickparvar_brain_tumor_mri_dataset_path
train_dir = USER_PATH + r'/Training/'
test_dir = USER_PATH + r'/Testing/'
CLASS_TYPES = ['pituitary', 'notumor', 'meningioma', 'glioma']
N_TYPES = len(CLASS_TYPES)

# Getting data using above function
train_paths, train_labels = get_data_labels(train_dir)
test_paths, test_labels = get_data_labels(test_dir)

# Printing traing and testing sample sizes
print('Training')
print(f'Number of Paths: {len(train_paths)}')
print(f'Number of Labels: {len(train_labels)}')
print('\nTesting')
print(f'Number of Paths: {len(test_paths)}')
print(f'Number of Labels: {len(test_labels)}')

_, ax = plt.subplots(ncols=3, figsize=(20, 14))

# Plotting training data types
class_counts = [len([x for x in train_labels if x == label]) for label in CLASS_TYPES]
print('Training Counts')
print(dict(zip(CLASS_TYPES, class_counts)))

ax[0].set_title('Dados de Treino')
ax[0].pie(
    class_counts,
    labels=[label.title() for label in CLASS_TYPES],
    colors=['#FAC500','#0BFA00', '#0066FA','#FA0000'],
    autopct=lambda p: '{:.2f}%\n{:,.0f}'.format(p, p * sum(class_counts) / 100),
    explode=tuple(0.01 for i in range(N_TYPES)),
    textprops={'fontsize': 20}
)

# Plotting testing data types
class_counts = [len([x for x in test_labels if x == label]) for label in CLASS_TYPES]
print('\nTesting Counts')
print(dict(zip(CLASS_TYPES, class_counts)))

ax[1].set_title('Dados de Teste')
ax[1].pie(
    class_counts,
    labels=[label.title() for label in CLASS_TYPES],
    colors=['#FAC500', '#0BFA00', '#0066FA', '#FA0000'],
    autopct=lambda p: '{:.2f}%\n{:,.0f}'.format(p, p * sum(class_counts) / 100),
    explode=tuple(0.01 for i in range(N_TYPES)),  # Explode the slices slightly for better visualization
    textprops={'fontsize': 20}  # Set the font size for the text on the pie chart
)

# Plotting distribution of train test split
ax[2].set_title('Split Treino/Teste')
ax[2].pie(
    [len(train_labels), len(test_labels)],
    labels=['Train','Test'],
    colors=['darkcyan', 'orange'],
    autopct=lambda p: '{:.2f}%\n{:,.0f}'.format(p, p * sum([len(train_labels), len(test_labels)]) / 100),
    explode=(0.1, 0),
    startangle=85,
    textprops={'fontsize': 20}
)


plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Junta os rótulos de treino e teste
all_labels = list(train_labels) + list(test_labels)

# Conta as ocorrências de cada classe
class_counts = [all_labels.count(label) for label in CLASS_TYPES]

print('Total Counts (Train+Test)')
print(dict(zip(CLASS_TYPES, class_counts)))

# Gráfico de pizza único
fig, ax = plt.subplots(figsize=(8, 6))
ax.set_title('Distribuição de Classes')
ax.pie(
    class_counts,
    labels=[label.title() for label in CLASS_TYPES],
    colors=['#FAC500','#0BFA00','#0066FA','#FA0000'],
    autopct=lambda p: '{:.2f}%\n{:,.0f}'.format(p, p * sum(class_counts) / 100),
    explode=tuple(0.02 for _ in CLASS_TYPES),
    textprops={'fontsize': 16}
)
ax.axis('equal')
plt.show()

# **PROBLEMAS ENCONTRADOS**

Foram reportados problemas com duplicatas nos datasets escolhidos, para isso deve ser feito um trabalho de identificação das duplicatas e tratamento para remoção das mesmas, de forma a não impactar no treinamento dos modelos.

# **Identificação das imagens duplicadas**
Esta parte do código deve ser feita diretamente no desktop pois ao fazer diretamente no colab a sessão se desconecta por ser versão gratuita e não poder utilizar dos recursos por completa.

In [ ]:
# from pathlib import Path  # Para manipulação de caminhos de arquivos
# import cv2                # OpenCV para processamento de imagem

# # Define o diretório base como o atual
# base_dir = Path.cwd()

# # Define os caminhos de Treino e Teste
# TRAIN_DIR = base_dir / "Train"
# TEST_DIR = base_dir / "Test"

# def compare_images(img1_path, img2_path, threshold=0.95):
#     """Compara duas imagens usando correlação de template."""

#     # Carrega imagens em escala de cinza
#     img1 = cv2.imread(str(img1_path), cv2.IMREAD_GRAYSCALE)
#     img2 = cv2.imread(str(img2_path), cv2.IMREAD_GRAYSCALE)

#     # Pula se o arquivo estiver corrompido ou não for encontrado
#     if img1 is None or img2 is None:
#         print(f"Aviso: Não foi possível ler {img1_path} ou {img2_path}")
#         return False, 0.0

#     # Redimensiona para um tamanho fixo
#     img1 = cv2.resize(img1, (224, 224))
#     img2 = cv2.resize(img2, (224, 224))

#     # Calcula o coeficiente de correlação normalizado (valor de -1 a 1)
#     correlation = cv2.matchTemplate(img1, img2, cv2.TM_CCOEFF_NORMED)[0,0]

#     # Retorna True se a correlação for maior que o limiar
#     return correlation > threshold, correlation

# # Encontra todos os arquivos .jpg dentro das subpastas
# train_paths = list(TRAIN_DIR.glob("*/*.jpg"))
# test_paths = list(TEST_DIR.glob("*/*.jpg"))

# # Imprime o status inicial
# print(f"Encontradas {len(train_paths)} imagens de treino.")
# print(f"Encontradas {len(test_paths)} imagens de teste.")
# print(f"Total de comparações a fazer: {len(train_paths) * len(test_paths)}")
# print("Iniciando varredura (isso pode levar muito tempo)...")

# # --- Loop Principal (Complexidade O(n*m) ---
# duplicates_list = []  # Armazena os pares duplicados
# counter = 0

# # Itera sobre cada imagem de treino
# for train_img in train_paths:
#     # Compara com cada imagem de teste
#     for test_img in test_paths:

#         is_duplicate, correlation_score = compare_images(train_img, test_img)

#         # Se forem similares o suficiente, registra
#         if is_duplicate:
#             counter += 1
#             print(f"Potencial duplicata #{counter}: {train_img.name} <-> {test_img.name} | Corr: {correlation_score:.4f}")
#             duplicates_list.append([str(train_img), str(test_img), correlation_score])

# print(f"\nComparação concluída. Encontradas {counter} duplicatas em potencial.")

# # --- Geração do Relatório ---
# output_file_path = base_dir / "relatorio_duplicatas.txt"

# try:
#     # Abre o arquivo de relatório (sobrescreve se existir)
#     with open(output_file_path, 'w') as f:
#         f.write(f"Total de Duplicatas em Potencial Encontradas: {counter}\n")
#         f.write("="*60 + "\n\n")

#         if not duplicates_list:
#             f.write("Nenhuma duplicata encontrada.\n")
#         else:
#             # Escreve o cabeçalho da tabela
#             f.write(f"{'Correlação':<12} | {'Imagem de Treino':<50} | {'Imagem de Teste'}\n")
#             f.write("-" * 100 + "\n")

#             # Escreve cada duplicata encontrada no arquivo
#             for item in duplicates_list:
#                 train_path, test_path, score = item[0], item[1], item[2]
#                 line = f"{score:<12.4f} | {train_path:<50} | {test_path}\n"
#                 f.write(line)

#     print(f"\nRelatório de duplicatas salvo com sucesso em: {output_file_path}")

# except Exception as e:
#     # Informa o usuário se houver erro ao salvar
#     print(f"\nOcorreu um erro ao salvar o arquivo: {e}")

# **Deletando as imagens duplicadas**
Após obter o arquivo que contém as duplicatas é necessário apagar essas imagens. Para isso, foi feito o código abaixo.

In [ ]:
import os, re, shutil, tempfile

# ===================== CONFIGURE AQUI =====================
# Raiz onde está o dataset ORIGINAL (pode ser somente-leitura, ex.: /kaggle/input/...)
dataset_root = masoudnickparvar_brain_tumor_mri_dataset_path

# Caminho do relatório .txt (pode estar em qualquer lugar)
report_file = gabrielrocha2_duplicatas_path + "/relatorio_duplicatas.txt"  # <-- ajuste

# Pasta espelho (gravável) para operar a limpeza
writable_mirror = "/content/clean_dataset"

# Por segurança, simulação primeiro
DRY_RUN = False

# (Opcional) fallback por prefixo Windows
USE_PREFIX_FALLBACK = True
WINDOWS_PREFIX = r"C:\Users\LSE\OneDrive\Documentos\Mestrado\TrabalhoAP"
# ==========================================================

# ---------- utilidades ----------
def is_dir_writable(path_dir: str) -> bool:
    try:
        os.makedirs(path_dir, exist_ok=True)
        fd, tmp = tempfile.mkstemp(dir=path_dir)
        os.close(fd)
        os.remove(tmp)
        return True
    except Exception:
        return False

# Regex para pegar sufixo a partir de Train/ ou Test/
pat_tt = re.compile(r'[/\\](Train|Test)[/\\].*', flags=re.IGNORECASE)

def to_dataset_relative(any_path: str) -> str | None:
    """Retorna 'Train/.../arquivo.jpg' ou 'Test/.../arquivo.jpg' a partir de um caminho Windows/Unix."""
    if not any_path:
        return None
    s = any_path.strip().strip('"').strip("'")
    m = pat_tt.search(s)
    if not m:
        return None
    rel = s[m.start()+1:].replace('\\', '/')
    return rel

def map_report_path(any_path: str, base_root: str) -> str | None:
    """Mapeia caminho do relatório para um arquivo dentro de base_root."""
    rel = to_dataset_relative(any_path)
    if rel:
        return os.path.join(base_root, rel)
    if USE_PREFIX_FALLBACK:
        s = any_path.strip().strip('"').strip("'").replace('\\', '/')
        win_pref = WINDOWS_PREFIX.replace('\\', '/').lower()
        if s.lower().startswith(win_pref):
            rel2 = s[len(win_pref):].lstrip('/\\')
            return os.path.join(base_root, rel2)
    return None

def is_noise_line(line: str) -> bool:
    l = line.strip()
    if not l: return True
    if set(l) <= set("-="): return True
    low = l.lower()
    if low.startswith("total de duplicatas") or low.startswith("correlação"):
        return True
    if "imagem de treino" in low and "imagem de teste" in low:
        return True
    return False

# ---------- prepara espelho gravável se necessário ----------
mirror_in_use = dataset_root
# if not is_dir_writable(dataset_root):
print(f"[INFO] Diretório somente-leitura detectado: {dataset_root}")
print(f"[INFO] Criando/atualizando espelho gravável em: {writable_mirror}")
# Copia o dataset inteiro uma única vez (se ainda não existir)
if not os.path.exists(writable_mirror) or not os.listdir(writable_mirror):
    shutil.copytree(dataset_root, writable_mirror, dirs_exist_ok=True)
    print("[OK] Espelho criado/copied.")
else:
    print("[OK] Espelho já existe — reutilizando.")
mirror_in_use = writable_mirror
# else:
#     print(f"[OK] Diretório gravável: {dataset_root} — operando direto nele.")

# ---------- processamento do relatório ----------
deleted_files_log = set()
files_deleted_count = 0
files_not_found_count = 0
lines_processed = 0
mapped_ok = 0
mapped_fail = 0

print(f"Relatório: {report_file}")
print(f"Operando em: {mirror_in_use}")
print(f"DRY_RUN: {DRY_RUN}")

if not os.path.isfile(report_file):
    raise FileNotFoundError(f"Relatório não encontrado: {report_file}")

with open(report_file, 'r', encoding='latin-1') as f:
    for raw in f:
        lines_processed += 1
        line = raw.rstrip('\n')
        if is_noise_line(line) or '|' not in line:
            continue

        parts = [p.strip() for p in line.split('|')]
        if len(parts) != 3:
            continue  # esperado: score | treino | teste

        score_str, train_win, test_win = parts
        if test_win.lower().startswith("imagem de teste"):
            continue

        target_path = map_report_path(test_win, mirror_in_use)
        if not target_path:
            mapped_fail += 1
            continue

        mapped_ok += 1
        if target_path in deleted_files_log:
            continue
        deleted_files_log.add(target_path)

        if os.path.exists(target_path):
            if DRY_RUN:
                print(f"[SIMULA EXCLUSÃO] {target_path}")
            else:
                try:
                    os.remove(target_path)
                    print(f"[EXCLUÍDO] {target_path}")
                    files_deleted_count += 1
                except OSError as e:
                    print(f"[ERRO AO EXCLUIR] {target_path}: {e}")
        else:
            print(f"[NÃO ENCONTRADO] {target_path}")
            files_not_found_count += 1

print("\n--- Resumo ---")
print(f"Linhas processadas: {lines_processed}")
print(f"Mapeamentos OK: {mapped_ok}")
print(f"Sem mapeamento: {mapped_fail}")
print(f"Arquivos EXCLUÍDOS: {files_deleted_count}")
print(f"Arquivos NÃO encontrados: {files_not_found_count}")
print("Quando estiver tudo certo, defina DRY_RUN = False para aplicar as exclusões.")


In [ ]:
import os
import shutil
import random
import glob
from pathlib import Path

# ===================== CONFIG BÁSICA (Colab) =====================
# Informe aqui possíveis locais do seu dataset (adicione mais caminhos se quiser).
CANDIDATE_BASE_DIRS = [
    "/content/clean_dataset",
    "/content/dataset",
    "/content/drive/MyDrive/clean_dataset",  # se o Drive estiver montado
]

OUTPUT_DIR = "/content/resplitted_dataset"
TRAIN_SPLIT_RATIO = 0.8
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

# Detecta BASE_DIR existente
BASE_DIR = None
for c in CANDIDATE_BASE_DIRS:
    if os.path.isdir(c):
        BASE_DIR = Path(c)
        break

if BASE_DIR is None:
    raise FileNotFoundError(
        "Não encontrei o dataset. Crie/copiei para um destes caminhos ou ajuste a lista:\n"
        + "\n".join(f" - {p}" for p in CANDIDATE_BASE_DIRS)
    )

print(f"BASE_DIR: {BASE_DIR}")
print(f"OUTPUT_DIR: {OUTPUT_DIR}")

# ===================== DETECÇÃO DE NOMES DAS PASTAS =====================
# Aceita Train/Training (ou variações de caixa) e Test/Testing
def pick_first_existing(parent, names):
    for n in names:
        p = os.path.join(parent, n)
        if os.path.isdir(p):
            return p
    return None

original_train_dir = pick_first_existing(BASE_DIR, ["Train", "train", "Training", "training"])
original_test_dir  = pick_first_existing(BASE_DIR, ["Test", "test", "Testing", "testing"])

if not original_train_dir and not original_test_dir:
    raise FileNotFoundError(
        f"Não achei pastas de treino/teste em {BASE_DIR}.\n"
        "Esperado algo como 'Train'/'Training' e/ou 'Test'/'Testing' com subpastas de classes."
    )

print("Origem detectada:")
print("  Train dir:", original_train_dir if original_train_dir else "(ausente)")
print("  Test  dir:", original_test_dir  if original_test_dir  else "(ausente)")

# ===================== LISTA DE CLASSES =====================
classes_in_train = set()
classes_in_test  = set()

if original_train_dir:
    classes_in_train = {d.name for d in os.scandir(original_train_dir) if d.is_dir()}
if original_test_dir:
    classes_in_test  = {d.name for d in os.scandir(original_test_dir)  if d.is_dir()}

all_classes = sorted(classes_in_train | classes_in_test)
if not all_classes:
    raise RuntimeError("Nenhuma subpasta de classe encontrada em Train/Training ou Test/Testing.")

print("Classes encontradas:", all_classes)

# ===================== CRIA SAÍDA =====================
new_train_dir = os.path.join(OUTPUT_DIR, "train")
new_test_dir  = os.path.join(OUTPUT_DIR, "test")
os.makedirs(new_train_dir, exist_ok=True)
os.makedirs(new_test_dir,  exist_ok=True)
for cls in all_classes:
    os.makedirs(os.path.join(new_train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(new_test_dir,  cls), exist_ok=True)

# ===================== FUNÇÕES ÚTEIS =====================
def safe_copy(src, dst_dir):
    """Copia sem sobrescrever: se nome já existir, cria _1, _2, ..."""
    base = os.path.basename(src)
    name, ext = os.path.splitext(base)
    dest = os.path.join(dst_dir, base)
    k = 1
    while os.path.exists(dest):
        dest = os.path.join(dst_dir, f"{name}_{k}{ext}")
        k += 1
    shutil.copy2(src, dest)

# ===================== SPLIT 80/20 =====================
print("\n--- Iniciando a separação 80/20 por classe ---")
total_train_all = 0
total_test_all  = 0
exts = ('*.jpg','*.jpeg','*.png','*.bmp','*.tif','*.tiff')

for cls in all_classes:
    print(f"\nProcessando classe: {cls}")
    imgs = []

    if original_train_dir:
        p_train = os.path.join(original_train_dir, cls)
        if os.path.isdir(p_train):
            for e in exts:
                imgs.extend(glob.glob(os.path.join(p_train, e)))

    if original_test_dir:
        p_test = os.path.join(original_test_dir, cls)
        if os.path.isdir(p_test):
            for e in exts:
                imgs.extend(glob.glob(os.path.join(p_test, e)))

    # Remove duplicatas de caminho e embaralha
    imgs = list(dict.fromkeys(imgs))
    n = len(imgs)
    if n == 0:
        print("  Nenhuma imagem. Pulando.")
        continue

    random.shuffle(imgs)
    split_point = int(n * TRAIN_SPLIT_RATIO)
    train_imgs = imgs[:split_point]
    test_imgs  = imgs[split_point:]

    print(f"  Total: {n} | Treino: {len(train_imgs)} | Teste: {len(test_imgs)}")

    dt = os.path.join(new_train_dir, cls)
    de = os.path.join(new_test_dir,  cls)

    for p in train_imgs:
        try:
            safe_copy(p, dt)
        except Exception as e:
            print(f"   [ERRO copiar treino] {p} -> {e}")

    for p in test_imgs:
        try:
            safe_copy(p, de)
        except Exception as e:
            print(f"   [ERRO copiar teste]  {p} -> {e}")

    total_train_all += len(train_imgs)
    total_test_all  += len(test_imgs)

print("\n--- Processo Concluído ---")
print(f"Total de imagens de TREINO copiadas: {total_train_all}")
print(f"Total de imagens de TESTE copiadas:  {total_test_all}")
print(f"Total geral de imagens processadas: {total_train_all + total_test_all}")
print(f"\nSeus novos dados estão prontos em: {OUTPUT_DIR}")


In [ ]:
# ----------------------------------------------------------------------
# IMPORTAÇÕES E CONFIGURAÇÃO DO AMBIENTE
# ----------------------------------------------------------------------
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, Callback
import time

import matplotlib.pyplot as plt
import numpy as np
import os
import random
from pathlib import Path
from google.colab import drive

import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import balanced_accuracy_score, roc_curve, auc
from sklearn.preprocessing import label_binarize



In [ ]:
# ----------------------------------------------------------------------
# REPRODUTIBILIDADE
# ----------------------------------------------------------------------
SEED = 129

def set_seeds(seed_value=SEED):
    """Fixa as sementes para reprodutibilidade."""
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    random.seed(seed_value)
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

set_seeds()

# Montar o Google Drive
# if not os.path.exists('/content/drive/MyDrive'):
#     drive.mount('/content/drive')
#     print("Google Drive montado.")
# else:
#     print("Google Drive já está montado.")

In [ ]:
# ----------------------------------------------------------------------
# DEFINIR CAMINHOS E PARÂMETROS
# ----------------------------------------------------------------------
base_dir = Path("/content/resplitted_dataset")
TRAIN_DIR = base_dir / "train"
TEST_DIR = base_dir / "test"

IMG_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 50
N_CLASSES = 4
IMG_SHAPE = IMG_SIZE + (3,)

In [ ]:
# ----------------------------------------------------------------------
# CARREGAR DADOS
# ----------------------------------------------------------------------

print("Carregando dataset de treino e validação...")
train_dataset = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    validation_split=0.2,    # 20% dos dados de treino irão para validação
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    validation_split=0.2,
    subset="validation",     # Pegando a outra parte dos dados
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

print("\nCarregando dataset de teste...")
test_dataset = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    shuffle=False
)

# Verificação das classes
class_names = train_dataset.class_names
print(f"\nClasses encontradas: {class_names}")
print(f"Número de classes: {len(class_names)}")

In [ ]:
# ----------------------------------------------------------------------
# VISUALIZAR UM EXEMPLO DE CADA CLASSE
# ----------------------------------------------------------------------

print("\nExibindo um exemplo de cada classe...")

# Configura a grade de plotagem (1 linha, 4 colunas)
plt.figure(figsize=(16, 4))

for i, class_name in enumerate(class_names):
    ax = plt.subplot(1, N_CLASSES, i + 1)

    # 1. Encontra o caminho para a subpasta da classe
    class_dir = TRAIN_DIR / class_name

    try:
        # 2. Pega o nome do PRIMEIRO arquivo de imagem nessa pasta
        sample_image_name = os.listdir(class_dir)[0]

        # 3. Monta o caminho completo para essa imagem
        sample_image_path = class_dir / sample_image_name

        # 4. Carrega a imagem
        img = tf.keras.utils.load_img(
            sample_image_path,
            target_size=IMG_SIZE
        )

        # 5. Mostra a imagem
        plt.imshow(img)
        plt.title(class_name)
        plt.axis("off")

    except Exception as e:
        # Se a pasta estiver vazia ou der um erro
        plt.title(f"{class_name}\n(Erro ao carregar)")
        plt.axis("off")
        print(f"Erro ao carregar imagem da classe {class_name}: {e}")

plt.tight_layout()
plt.show()

print("Salvando caminhos dos arquivos de teste...")
test_filepaths = test_dataset.file_paths

In [ ]:
# ----------------------------------------------------------------------
# OTIMIZAR O CARREGAMENTO (PERFORMANCE)
# ----------------------------------------------------------------------
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# ----------------------------------------------------------------------
# CRIAR CAMADA DE "DATA AUGMENTATION"
# ----------------------------------------------------------------------
data_augmentation = Sequential(
    [
        layers.RandomFlip("horizontal", input_shape=IMG_SHAPE),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ],
    name="data_augmentation"
)

In [ ]:
# ----------------------------------------------------------------------
# CONSTRUINDO O MODELO CNN
# ----------------------------------------------------------------------
model = Sequential([
    # 1. Camada de "Aumento de Dados"
    data_augmentation,

    # 2. Normalização: Mapeia pixels de [0, 255] para [0, 1]
    layers.Rescaling(1./255),

    # 3. Bloco Convolucional 1
    layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D(),

    # 4. Bloco Convolucional 2
    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D(),

    # 5. Bloco Convolucional 3
    layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D(),

    # 6. Bloco Convolucional 4
    layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D(),

    # 7. Camada de "Dropout"
    layers.Dropout(0.2),

    # 8. Achatar (Flatten)
    layers.Flatten(),

    # 9. Camada Densa
    layers.Dense(256, activation='relu'),

    # 10. Camada de Saída
    layers.Dense(N_CLASSES, activation='softmax')
])

In [ ]:
# ----------------------------------------------------------------------
# CALLBACK: REGISTRAR TEMPO POR ÉPOCA
# ----------------------------------------------------------------------
class TimeHistory(Callback):
    def on_train_begin(self, logs={}):
        self.epoch_times = []

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs={}):
        epoch_time = time.time() - self.epoch_start_time
        self.epoch_times.append(epoch_time)
        print(f" - Tempo da Época: {epoch_time:.2f}s")

time_callback = TimeHistory()

In [ ]:
# ----------------------------------------------------------------------
# COMPILANDO O MODELO
# ----------------------------------------------------------------------
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='roc_auc')]
)

# Callback para parar o treino se a performance não melhorar
early_stopping = EarlyStopping(
    monitor='val_loss', # Monitorar a perda na validação
    patience=10,        # Parar após 10 épocas sem melhora
    verbose=1,
    restore_best_weights=True # Salvar o melhor modelo
)

model.summary()

In [ ]:
# ----------------------------------------------------------------------
# TREINANDO O MODELO
# ----------------------------------------------------------------------

print("\nIniciando o treinamento...")

history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=EPOCHS,
    callbacks=[early_stopping, time_callback]
)

print("Treinamento concluído.")

In [ ]:
# ----------------------------------------------------------------------
# AVALIAÇÃO DO MODELO COM DADOS DE TESTE
# ----------------------------------------------------------------------

print("\nIniciando avaliação no dataset de TESTE...")
loss, accuracy, roc_auc = model.evaluate(test_dataset)

print(f"\n--- Resultados da Avaliação de Teste ---")
print(f"Perda (Loss) no Teste: {loss:.4f}")
print(f"Acurácia no Teste: {accuracy * 100:.2f}%")
print(f"ROC-AUC no Teste: {roc_auc:.4f}")

print("\n--- Custo Computacional ---")
tempos_por_epoca = time_callback.epoch_times
print(f"Hardware: (Executando no Google Colab)")
print(f"Tempo médio por época: {np.mean(tempos_por_epoca):.2f}s")
print(f"Tempo total de treino: {np.sum(tempos_por_epoca):.2f}s")

In [ ]:
# ----------------------------------------------------------------------
# PLOTAR OS RESULTADOS DO TREINAMENTO
# ----------------------------------------------------------------------

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(12, 6))

# Gráfico de Acurácia
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Acurácia (Treino)')
plt.plot(epochs_range, val_acc, label='Acurácia (Validação)')
plt.legend(loc='lower right')
plt.title('Acurácia de Treino vs. Validação')

# Gráfico de Perda (Loss)
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Perda (Treino)')
plt.plot(epochs_range, val_loss, label='Perda (Validação)')
plt.legend(loc='upper right')
plt.title('Perda de Treino vs. Validação')

plt.show()

In [ ]:
# ----------------------------------------------------------------------
# MATRIZ DE CONFUSÃO
# ----------------------------------------------------------------------

print("\nGerando Matriz de Confusão para o Set de TESTE...")

# Obter as previsões
y_pred_probs = model.predict(test_dataset)
y_pred_classes = np.argmax(y_pred_probs, axis=1) # Converter probabilidades para classes

# Obter as classes reais do test_dataset
y_true_classes = []
for images, labels in test_dataset:
    y_true_classes.extend(np.argmax(labels.numpy(), axis=1))

# Gerar a matriz de confusão
cm = confusion_matrix(y_true_classes, y_pred_classes)

# Plotar a matriz de confusão
plt.figure(figsize=(7, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Classe Predita')
plt.ylabel('Classe Real')
plt.title('Matriz de Confusão do Set de TESTE')
plt.show()

# Imprimir o relatório de classificação completo (F1-score, Precision, Recall)
print("\nRelatório de Classificação (Set de TESTE):")
print(classification_report(y_true_classes, y_pred_classes, target_names=class_names))

In [ ]:
# ----------------------------------------------------------------------
# MÉTRICAS: BALANCED ACCURACY E CURVAS ROC
# ----------------------------------------------------------------------

# 1. Balanced Accuracy
bal_acc = balanced_accuracy_score(y_true_classes, y_pred_classes)
print(f"Acurácia Balanceada (Balanced Accuracy): {bal_acc:.4f}")

# --- 2. Curvas ROC e ROC-AUC (One-vs-Rest) ---

# Binarizar os rótulos reais (y_true)
y_true_binarized = label_binarize(y_true_classes, classes=range(N_CLASSES))

# Dicionários para guardar os valores de FPR, TPR e AUC
fpr = dict()
tpr = dict()
roc_auc = dict()

# Calcular FPR, TPR e AUC para cada classe
for i in range(N_CLASSES):
    fpr[i], tpr[i], _ = roc_curve(y_true_binarized[:, i], y_pred_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plotar as curvas ROC
plt.figure(figsize=(10, 8))
colors = ['blue', 'red', 'green', 'orange']
for i, color in zip(range(N_CLASSES), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label=f'Classe {class_names[i]} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2) # Linha de chance aleatória
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taxa de Falsos Positivos (FPR)')
plt.ylabel('Taxa de Verdadeiros Positivos (TPR)')
plt.title('Curvas ROC Multi-Classe (One-vs-Rest)')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# ----------------------------------------------------------------------
# ANÁLISE DE ERRO (AMOSTRAS COMENTADAS)
# ----------------------------------------------------------------------

# Pegar todos os caminhos de arquivo do set de teste
#test_filepaths = test_dataset.file_paths

# Encontrar os índices dos erros
errors = np.where(y_pred_classes != y_true_classes)[0]
print(f"\nTotal de erros no set de teste: {len(errors)} de {len(y_true_classes)} amostras.")

if len(errors) > 0:
    print("Exibindo as primeiras 9 amostras classificadas incorretamente...")

    plt.figure(figsize=(12, 12))
    for i, error_index in enumerate(errors[:9]):
        plt.subplot(3, 3, i + 1)

        # Carregar a imagem original
        img = tf.keras.utils.load_img(test_filepaths[error_index], target_size=IMG_SIZE)
        plt.imshow(img)

        real_label = class_names[y_true_classes[error_index]]
        pred_label = class_names[y_pred_classes[error_index]]

        plt.title(f"Real: {real_label}\nPredito: {pred_label}", color='red')
        plt.axis('off')
    plt.tight_layout()
    plt.show()
else:
    print("O modelo acertou todas as amostras do set de teste!")

# **==============================================================================================================**

# **=======================================YOLO=======================================**

# **==============================================================================================================**

# **INSTALAÇÃO DE LIBS E DE EXTENSÕES** #

In [ ]:
import kagglehub
masoudnickparvar_brain_tumor_mri_dataset_path = kagglehub.dataset_download('masoudnickparvar/brain-tumor-mri-dataset')
ahmedsorour1_mri_for_brain_tumor_with_bounding_boxes_path = kagglehub.dataset_download('ahmedsorour1/mri-for-brain-tumor-with-bounding-boxes')
gabrielrocha2_duplicatas_path = kagglehub.dataset_download('gabrielrocha2/duplicatas')

print('Data source import complete.')

In [ ]:
# Limpar o trio para evitar sobras binárias
# !pip uninstall -y numpy scipy scikit-learn

# # Instalar um conjunto compatível e moderno
# !pip install --no-cache-dir --force-reinstall \
#   "numpy==2.0.2" "scipy==1.13.1" "scikit-learn==1.5.2"

In [ ]:
import numpy, scipy, sklearn
from scipy._lib import _util
print("NumPy:", numpy.__version__, "| SciPy:", scipy.__version__, "| Sklearn:", sklearn.__version__)
print("has np_vecdot?", hasattr(_util, "np_vecdot"))  # deve ser True

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
print("✅ sklearn OK")

In [ ]:

# !pip install --no-cache-dir --force-reinstall "scipy==1.13.1"


In [ ]:
#!pip uninstall -y numpy scipy scikit-learn ml_dtypes tensorflow tensorflow-text tensorflow-decision-forests tf-keras mkl-umath mkl-random mkl-fft numba ydata-profiling
#!pip install --no-cache-dir --force-reinstall "numpy==1.26.4" "scipy==1.10.1" "scikit-learn==1.3.2"
#!pip install -U ultralytics opencv-python matplotlib
# !pip install -U ultralytics opencv-python scikit-learn matplotlib
import os, shutil, json, glob, random
from pathlib import Path
import numpy as np
import cv2


from ultralytics import YOLO  # supports yolo11 and yolo8 models

# All necessary imports
import numpy as np
import pandas as pd
import os
import glob
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
import ipywidgets as widgets
import io
from PIL import Image
from sklearn.model_selection import train_test_split
import cv2
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder


# **CONFIGURAÇÕES DO AMBIENTE** #

In [ ]:
import platform
import torch
import psutil

print("=== CONFIGURAÇÃO DE AMBIENTE ===")
print(f"Sistema Operacional: {platform.platform()}")
print(f"Python: {platform.python_version()}")
print(f"CPU: {platform.processor()} ({psutil.cpu_count(logical=True)} núcleos)")
print(f"Memória RAM: {round(psutil.virtual_memory().total / 1e9, 2)} GB")
print()

# PyTorch
print("PyTorch versão:", torch.__version__)
if torch.cuda.is_available():
    print("CUDA disponível ✅")
    print("GPU ativa:", torch.cuda.get_device_name(0))
    print("Total de GPUs:", torch.cuda.device_count())
    print("Versão CUDA:", torch.version.cuda)
else:
    print("CUDA não detectada ❌")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\nDispositivo em uso: {DEVICE}")

# **UNIÃO DE TODAS AS IMAGENS E LABELS EM UM SÓ DIRETÓRIO PARA REALIZAR O SPLIT CORRETAMENTE** #

In [ ]:
import os
import shutil

# === CONFIGURAÇÕES ===
# Caminho do dataset original (mude conforme o seu dataset Kaggle)
# Exemplo: "/kaggle/input/brain-tumor-dataset"
# BASE_DIR = "/kaggle/input/mri-for-brain-tumor-with-bounding-boxes"
BASE_DIR = ahmedsorour1_mri_for_brain_tumor_with_bounding_boxes_path

# Caminho onde será criada a nova estrutura YOLO
# OUTPUT_DIR = "/kaggle/working/mri_brain_dataset_yolo_unified"
OUTPUT_DIR = "/content/mri_brain_dataset_yolo_unified"

# Cria as novas pastas únicas para imagens e labels
os.makedirs(os.path.join(OUTPUT_DIR, "images"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "labels"), exist_ok=True)

# Pastas de origem (train e val originais)
SOURCE_SETS = ["Train", "Val"]

# === COPIA E RENOMEIA OS ARQUIVOS ===
for source_split in SOURCE_SETS:
    split_path = os.path.join(BASE_DIR, source_split)

    for class_name in os.listdir(split_path):
        class_path = os.path.join(split_path, class_name)
        images_path = os.path.join(class_path, "images")
        labels_path = os.path.join(class_path, "labels")

        if not os.path.isdir(images_path) or not os.path.isdir(labels_path):
            continue

        # Copia as imagens
        for img_file in os.listdir(images_path):
            src = os.path.join(images_path, img_file)
            dst = os.path.join(OUTPUT_DIR, "images", f"{class_name}_{img_file}")
            shutil.copy2(src, dst)

        # Copia os labels
        for label_file in os.listdir(labels_path):
            src = os.path.join(labels_path, label_file)
            dst = os.path.join(OUTPUT_DIR, "labels", f"{class_name}_{label_file}")
            shutil.copy2(src, dst)

print("✅ Dataset convertido e unificado com sucesso!")
print("Nova estrutura criada em:", OUTPUT_DIR)
print()
print("dataset_yolo/")
print(" ├─ images/")
print(" └─ labels/")


# **APAGANDO DUPLICATAS PARA NÃO ENVIESAR O MODELO** #

In [ ]:
import os, re, shutil
from pathlib import Path

# ===== CONFIG =====
# DATASET_ROOT = Path("/kaggle/working/mri_brain_dataset_yolo_unified")      # dataset unificado
DATASET_ROOT = Path(OUTPUT_DIR)
IMAGES_DIR   = DATASET_ROOT / "images"
LABELS_DIR   = DATASET_ROOT / "labels"

# DUPES_FILE   = Path("/kaggle/input/duplicatas/relatorio_duplicatas.txt")
DUPES_FILE   = Path(gabrielrocha2_duplicatas_path + "/relatorio_duplicatas.txt")
CORR_MIN     = 0.5 # coloquei treshold mas todos os relatados no txt realmente são duplicados (correlação maior de 95%)
DRY_RUN      = False # se DRY_RUN false apaga no dataset, se true apenas printa os duplicados
QUARANTINE   = None # se quiser isolar passar o path para o diretorio


# ===== HELPERS =====
def basename_win(path_str: str) -> str:
    """Extrai o basename de um path Windows/Unix (\ ou /)."""
    return re.split(r"[\\/]+", path_str.strip())[-1]

def parse_dupe_lines(text: str):
    """Lê linhas 'corr | caminhoA | caminhoB' → [{'corr':float,'a':str,'b':str}, ...]."""
    items = []
    for ln in text.splitlines():
        t = ln.strip()
        if not t or t.startswith("Total de") or set(t) in ({'='}, {'-'}):
            continue
        parts = [p.strip() for p in t.split("|")]
        if len(parts) != 3:
            continue
        try:
            corr = float(parts[0].replace(",", "."))
        except ValueError:
            continue
        items.append({"corr": corr, "a": parts[1], "b": parts[2]})
    return items

def build_image_index(images_dir: Path):
    """Cria índice {basename_lower: Path} e uma lista completa p/ fallback por sufixo."""
    index = {}
    all_paths = []
    # pega somente arquivos na pasta images (sem recursão, já que a estrutura está plana)
    for p in images_dir.iterdir():
        if p.is_file() and p.suffix.lower() in {".jpg",".jpeg",".png",".bmp",".tif",".tiff"}:
            index[p.name.lower()] = p
            all_paths.append(p)
    return index, all_paths

def find_image_by_basename(basename: str, idx: dict, all_paths: list[Path]) -> Path | None:
    """1) match exato (case-insensitive); 2) fallback: termina com esse basename (caso tenha prefixo)."""
    key = basename.lower()
    if key in idx:
        return idx[key]
    for p in all_paths:
        if p.name.lower().endswith(key):
            return p
    return None

def label_for_image(img_path: Path) -> Path:
    return LABELS_DIR / f"{img_path.stem}.txt"

def delete_pair(img_path: Path):
    lbl = label_for_image(img_path)
    if DRY_RUN:
        print(f"[DRY] DEL IMG: {img_path}")
        print(f"[DRY] DEL LBL: {lbl} {'(não existe)' if not lbl.exists() else ''}")
        return
    if QUARANTINE:
        q_img = QUARANTINE / "images" / img_path.name
        q_lbl = QUARANTINE / "labels" / lbl.name
        q_img.parent.mkdir(parents=True, exist_ok=True)
        q_lbl.parent.mkdir(parents=True, exist_ok=True)
        shutil.move(str(img_path), str(q_img))
        if lbl.exists(): shutil.move(str(lbl), str(q_lbl))
        print(f"[MOVE] {img_path.name} -> quarentena")
        return
    img_path.unlink(missing_ok=True)
    if lbl.exists(): lbl.unlink()
    print(f"[OK] Removido: {img_path.name}")

# ===== SANITY CHECK =====
assert IMAGES_DIR.exists(), f"Não encontrado {IMAGES_DIR}"
assert LABELS_DIR.exists(), f"Não encontrado {LABELS_DIR}"
assert DUPES_FILE.exists(), f"Não encontrado na lista: {DUPES_FILE}"

print("IMAGES_DIR:", IMAGES_DIR)
print("LABELS_DIR:", LABELS_DIR)

img_index, img_all = build_image_index(IMAGES_DIR)
print(f"Imagens indexadas: {len(img_index)}")
print("Exemplos:", [p.name for p in img_all[:5]])

# ===== PROCESSAMENTO =====
items = parse_dupe_lines(DUPES_FILE.read_text(encoding="utf-8", errors="ignore"))
eligible = [r for r in items if r["corr"] >= CORR_MIN]
print(f"Duplicatas elegíveis (corr >= {CORR_MIN}): {len(eligible)}")

deleted = 0
not_found = 0

for rec in eligible:
    a_base = basename_win(rec["a"])
    b_base = basename_win(rec["b"])

    a_img = find_image_by_basename(a_base, img_index, img_all)
    b_img = find_image_by_basename(b_base, img_index, img_all)

    # regra simples: se os dois existem, apaga o B; se só um existe, apaga o que existe
    target = b_img or a_img
    if target is None:
        not_found += 1
        print(f"[MISS] Não encontrados: {a_base} | {b_base}")
        continue

    delete_pair(target)
    deleted += 1

print("\n===== RESUMO =====")
print(f"Apagadas (ou DRY): {deleted}")
print(f"Não localizadas: {not_found}")
print("Modo:", "DRY-RUN (sem apagar)" if DRY_RUN else ("QUARENTENA" if QUARANTINE else "APAGANDO"))

In [ ]:
# ===== Consistência YOLO + remoção de órfãos e itens com issues =====
import os, csv, shutil
from pathlib import Path

# === CONFIGS ===
#DATASET_ROOT = Path("/kaggle/working/dataset_yolo")  # contém images/ e labels/
IMAGES_DIR   = DATASET_ROOT / "images"
LABELS_DIR   = DATASET_ROOT / "labels"

IMAGE_EXTS   = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}
CLASSES      = ["glioma","meningioma","no_tumor","pituitary"] # erro na ordem das classes, refazer

# Ações (escolha)
DRY_RUN               = False   # primeiro rodando em True para revisar
DELETE_ORPHAN_LABELS  = True  # apaga labels sem imagem
DELETE_ORPHAN_IMAGES  = True  # apaga imagens sem label
DELETE_ISSUE_ITEMS    = True   # apaga imagem + label quando houver issue no label
QUARANTINE            = None   # ex.: Path("/kaggle/working/_quarentena") para mover ao invés de apagar

# Saídas
# OUT_DIR = Path("/kaggle/working/yolo_checks")
OUT_DIR = Path("/content/yolo_checks")
OUT_DIR.mkdir(parents=True, exist_ok=True)
REPORT_CSV          = OUT_DIR / "dataset_yolo_report.csv"
ORPHAN_IMAGES_TXT   = OUT_DIR / "images_sem_label.txt"
ORPHAN_LABELS_TXT   = OUT_DIR / "labels_sem_imagem.txt"
ISSUE_STEMS_TXT     = OUT_DIR / "stems_com_issues.txt"

# === Sanity ===
assert IMAGES_DIR.exists(), f"Não encontrado {IMAGES_DIR}"
assert LABELS_DIR.exists(), f"Não encontrado {LABELS_DIR}"

# === Indexação (por stem) ===
def list_images(images_dir: Path):
    return {p.stem: p for p in images_dir.iterdir()
            if p.is_file() and p.suffix.lower() in IMAGE_EXTS}

def list_labels(labels_dir: Path):
    return {p.stem: p for p in labels_dir.iterdir()
            if p.is_file() and p.suffix.lower() == ".txt"}

img_idx = list_images(IMAGES_DIR)
lbl_idx = list_labels(LABELS_DIR)

imgs_set = set(img_idx.keys())
lbls_set = set(lbl_idx.keys())

# === Órfãos ===
images_without_label = sorted(imgs_set - lbls_set)
labels_without_image = sorted(lbls_set - imgs_set)

# === Validação de conteúdo YOLO ===
def parse_line(line):
    # formato: class cx cy w h [ignora resto]
    parts = line.strip().split()
    if len(parts) < 5:
        return {"ok": False, "err": "colunas<5"}
    try:
        cls = int(float(parts[0]))
        cx, cy, w, h = map(float, parts[1:5])
    except Exception:
        return {"ok": False, "err": "parse_float/int"}

    if not (0 <= cls < len(CLASSES)):
        return {"ok": False, "err": f"class_out_of_range:{cls}"}
    for v in (cx, cy, w, h):
        if not (0.0 <= v <= 1.0):
            return {"ok": False, "err": "coords_out_of_[0,1]"}
    if w <= 0 or h <= 0:
        return {"ok": False, "err": "w/h<=0"}
    return {"ok": True}

issues = []            # linhas para CSV
issue_stems = set()    # stems com algum problema

for stem in sorted(imgs_set & lbls_set):
    lbl_path = lbl_idx[stem]
    text = lbl_path.read_text(encoding="utf-8", errors="ignore")
    lines = [ln for ln in text.splitlines() if ln.strip()]

    if len(lines) == 0:
        issues.append({"stem": stem, "type": "label_empty", "detail": "arquivo .txt vazio"})
        issue_stems.add(stem)
        continue

    for i, ln in enumerate(lines, start=1):
        res = parse_line(ln)
        if not res["ok"]:
            issues.append({"stem": stem, "type": "label_invalid", "detail": f"linha {i}: {res['err']}"})
            issue_stems.add(stem)

# === Helpers de mover/apagar ===
def do_remove(p: Path, subdir: str):
    if DRY_RUN:
        print(f"[DRY] remover: {p}")
        return
    if QUARANTINE:
        dst = QUARANTINE / subdir / p.name
        dst.parent.mkdir(parents=True, exist_ok=True)
        shutil.move(str(p), str(dst))
        print(f"[MOVE] {p} -> {dst}")
    else:
        p.unlink(missing_ok=True)
        print(f"[DEL] {p}")

def remove_image_and_label(stem: str):
    img = img_idx.get(stem)
    lbl = lbl_idx.get(stem)
    if img is None and lbl is None:
        print(f"[SKIP] {stem}: nem imagem nem label presentes.")
        return
    if img is not None:
        do_remove(img, "images")
        # após remover, também retire do índice para evitar dupla ação
        img_idx.pop(stem, None)
    if lbl is not None:
        do_remove(lbl, "labels")
        lbl_idx.pop(stem, None)

# === Ações: órfãos ===
if DELETE_ORPHAN_LABELS:
    for stem in labels_without_image:
        do_remove(lbl_idx[stem], "labels")

if DELETE_ORPHAN_IMAGES:
    for stem in images_without_label:
        do_remove(img_idx[stem], "images")

# === Ações: itens com issues (remove imagem + label) ===
if DELETE_ISSUE_ITEMS and issue_stems:
    print(f"\n=== Removendo itens com issues (total: {len(issue_stems)}) ===")
    for stem in sorted(issue_stems):
        remove_image_and_label(stem)

# === Relatórios ===
# CSV de issues
with REPORT_CSV.open("w", newline="", encoding="utf-8") as f:
    w = csv.DictWriter(f, fieldnames=["stem","type","detail"])
    w.writeheader()
    for row in issues:
        w.writerow(row)

# Listas de órfãos e issues
ORPHAN_IMAGES_TXT.write_text("\n".join(images_without_label), encoding="utf-8")
ORPHAN_LABELS_TXT.write_text("\n".join(labels_without_image), encoding="utf-8")
ISSUE_STEMS_TXT.write_text("\n".join(sorted(issue_stems)), encoding="utf-8")

# === Resumo ===
print("\n===== RESUMO =====")
print(f"Imagens totais: {len(list_images(IMAGES_DIR))} (após remoções em memória)")
print(f"Labels  totais: {len(list_labels(LABELS_DIR))} (após remoções em memória)")
print(f"Imagens SEM label: {len(images_without_label)}  -> {ORPHAN_IMAGES_TXT}")
print(f"Labels  SEM imagem: {len(labels_without_image)}  -> {ORPHAN_LABELS_TXT}")
print(f"Stems com issues: {len(issue_stems)}           -> {ISSUE_STEMS_TXT}")
print(f"Relatório detalhado: {REPORT_CSV}")
print("Modo:", "DRY-RUN (sem apagar/mover)" if DRY_RUN else ("QUARENTENA" if QUARANTINE else "APAGANDO"))

# **SPLIT CONFIGURÁVEL FEITO PARA TRAIN VAL E TEST, NO FORMATO EM QUE O YOLO ESPERA** #

In [ ]:
import os
import random
import shutil

# === CONFIGURAÇÕES ===
# DATASET_DIR = "/kaggle/working/mri_brain_dataset_yolo_unified"  # caminho do dataset unificado
DATASET_DIR = DATASET_ROOT
# OUTPUT_DIR = "/kaggle/working/mri_brain_dataset_yolo_split"  # onde será criado o split final
OUTPUT_DIR = "/content/mri_brain_dataset_yolo_split"

# Proporções do split (soma deve ser 1.0)
TRAIN_RATIO = 0.7
VAL_RATIO = 0.2
TEST_RATIO = 0.1

# Cria pastas de destino no formato YOLO
for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(OUTPUT_DIR, "images", split), exist_ok=True)
    os.makedirs(os.path.join(OUTPUT_DIR, "labels", split), exist_ok=True)

# Lista todas as imagens disponíveis
all_images = [f for f in os.listdir(os.path.join(DATASET_DIR, "images")) if f.endswith((".jpg", ".png", ".jpeg"))]
all_images.sort()

# Embaralha para aleatoriedade
random.shuffle(all_images)

# Calcula índices de divisão
n_total = len(all_images)
n_train = int(n_total * TRAIN_RATIO)
n_val = int(n_total * VAL_RATIO)
n_test = n_total - n_train - n_val

train_files = all_images[:n_train]
val_files = all_images[n_train:n_train + n_val]
test_files = all_images[n_train + n_val:]

# Função para copiar arquivos de imagem e label
def copy_split(files, split_name):
    for img_file in files:
        label_file = os.path.splitext(img_file)[0] + ".txt"
        img_src = os.path.join(DATASET_DIR, "images", img_file)
        lbl_src = os.path.join(DATASET_DIR, "labels", label_file)

        img_dst = os.path.join(OUTPUT_DIR, "images", split_name, img_file)
        lbl_dst = os.path.join(OUTPUT_DIR, "labels", split_name, label_file)

        # Copia imagem
        shutil.copy2(img_src, img_dst)

        # Copia label (se existir)
        if os.path.exists(lbl_src):
            shutil.copy2(lbl_src, lbl_dst)

# Copia os arquivos
copy_split(train_files, "train")
copy_split(val_files, "val")
copy_split(test_files, "test")

print("✅ Split concluído com sucesso!")
print(f"Total de imagens: {n_total}")
print(f" - Treino: {len(train_files)}")
print(f" - Validação: {len(val_files)}")
print(f" - Teste: {len(test_files)}")
print()
print("Estrutura criada em:", OUTPUT_DIR)
print("""
dataset_split/
├─ images/
│   ├─ train/
│   ├─ val/
│   └─ test/
└─ labels/
    ├─ train/
    ├─ val/
    └─ test/
""")


# **INCLUSÃO DE PARAMETROS E DIRETÓRIOS** #

In [ ]:
# ==== PROJECT ROOTS ====
ROOT = "/content"
PROJECT_DIR = Path(ROOT) / "brain_tumor_pipeline"
PROJECT_DIR.mkdir(parents=True, exist_ok=True)

# TODO: point these to your extracted datasets (Kaggle/Figshare/Harvard Medical)
# Expecting a YOLO-style detection dataset for step 4 (images + labels in YOLO txt format).
# DETECTION_DATA_ROOT = Path('/kaggle/working/mri_brain_dataset_yolo_split' )  # e.g., .../BrainTumorYolo
DETECTION_DATA_ROOT = Path(OUTPUT_DIR)
#  ├─ images/
#  │    ├─ train/*.jpg (or .png)
#  │    ├─ val/*.jpg
#  │    └─ test/*.jpg
#  └─ labels/
#       ├─ train/*.txt
#       ├─ val/*.txt
#       └─ test/*.txt
# YOLO txt: class cx cy w h (normalized)

# Class names (index order must match the YOLO labels)
#CLASSES = ["glioma", "meningioma", "pituitary", "no_tumor"]

# ==== MODEL / TRAINING PARAMS ====
IMG_SIZE_CNN = (150, 150)     # (Resize, Preprocess)
BATCH_SIZE = 32
EPOCHS_CNN = 25
RANDOM_SEED = 42

CONF_THRES = 0.25   # detection confidence threshold
IOU_THRES = 0.5     # NMS threshold


# **SALVA CONFIGURAÇÃO DO YOLO** #

In [ ]:
def write_yolo_yaml(root, yaml_path, class_names):
    # garante que 'root' e 'yaml_path' sejam Path mesmo que venham como string
    root = Path(root)
    yaml_path = Path(yaml_path)

    text = f"""path: {root.resolve()}
train: images/train
val: images/val
test: images/test

names:
"""
    for i, name in enumerate(class_names):
        text += f"  {i}: {name}\n"

    yaml_path.write_text(text)
    print(f"[YOLO] YAML gerado em: {yaml_path}")

YOLO_DATA_YAML = PROJECT_DIR/"brain_tumor_yolo.yaml"
write_yolo_yaml(OUTPUT_DIR, YOLO_DATA_YAML, CLASSES)
print(YOLO_DATA_YAML.read_text())


# **TREINAMENTO DO MODELO** #

In [ ]:
# Train YOLO detector
yolo = YOLO("yolo11n.pt") #testar com modelo de classificação
results = yolo.train(
    data=str(YOLO_DATA_YAML),
    epochs=50,
    imgsz=640,
    batch=16,
    optimizer="Adam",
    lr0=1e-3,
    weight_decay=5e-4,
    patience=10,
    device=0 if DEVICE=="cuda" else "cpu",
    verbose=True
)

# **SALVA MELHORES RESULTADOS DO MODELO** #

In [ ]:

# Best weights path (Ultralytics saves to runs/detect/train*/weights/best.pt)
best_weights = Path(yolo.trainer.best) if hasattr(yolo, "trainer") else None
if best_weights is None or not best_weights.exists():
    # try default save path format
    candidates = list(Path("runs/detect").glob("*/weights/best.pt"))
    best_weights = candidates[-1] if candidates else None


# **UTILIZA MELHORES RESULTADOS** #

In [ ]:
assert best_weights is not None and best_weights.exists(), "Could not locate trained YOLO weights."
print(f"[YOLO] Using weights: {best_weights}")
detector = YOLO(str(best_weights))

# **VISUALIZAÇÃO DOS GRÁFICOS E INFORMAÇÕES QUE O PROPRIO YOLO GERA COMO HISTORICO** #

In [ ]:
# Gera/regera as métricas de detecção (inclui mAP, PR/F1/P/R e confusion_matrix.png)
val_res = detector.val(data="/content/brain_tumor_pipeline/brain_tumor_yolo.yaml", split="test", imgsz=640, conf=0.25, iou=0.5, plots=True, verbose=False)
print(val_res.results_dict)  # mAP50, mAP50-95, precision, recall etc.

In [ ]:
from pathlib import Path
from IPython.display import display, Image

runs_root = Path("/content/runs/detect")
cands = sorted(runs_root.glob("val*"), key=lambda p: p.stat().st_mtime) or \
        sorted(runs_root.glob("train*"), key=lambda p: p.stat().st_mtime)
assert cands, "Nenhum run encontrado em runs/detect/"
run = cands[-1]
print("Run selecionado:", run)

imgs = [
    run/"results.png",           # curvas de treino/val (loss, mAP)
    run/"F1_curve.png",
    run/"PR_curve.png",
    run/"P_curve.png",
    run/"R_curve.png",
    run/"confusion_matrix.png",  # matriz de confusão (detecção)
    run/"labels.jpg",
    run/"val_batch0_pred.jpg",
    run/"val_batch1_pred.jpg",
    run/"val_batch2_pred.jpg",
]
for p in imgs:
    if p.exists():
        print(p.name)
        display(Image(filename=str(p)))

# **TESTA E GERA OS PARAMETROS DE AVALIAÇÃO, BEM COMO A MATRIZ DE CONFUSÃO** #

In [ ]:
import glob
from sklearn.metrics import f1_score, balanced_accuracy_score, roc_auc_score, classification_report, confusion_matrix
import json

ID_NO_TUMOR = CLASSES.index("no_tumor")

def read_yolo_label(txt_path: Path):
    if not txt_path.exists(): return []
    out = []
    for ln in txt_path.read_text().splitlines():
        p = ln.split()
        if len(p) < 5: continue
        c = int(float(p[0])); cx,cy,w,h = map(float, p[1:5])
        out.append((c,cx,cy,w,h))
    return out

def img_true_class(img_path: Path):
    lab = Path(OUTPUT_DIR)/"labels"/"test"/(img_path.stem + ".txt")
    boxes = read_yolo_label(lab)
    if not boxes: return ID_NO_TUMOR
    # maior área normalizada
    areas = [b[3]*b[4] for b in boxes]
    return boxes[int(np.argmax(areas))][0]

test_images = sorted(list((Path(OUTPUT_DIR)/"images"/"test").glob("*.jpg")) + list((Path(OUTPUT_DIR)/"images"/"test").glob("*.png")))
y_true, y_pred, y_prob = [], [], []

for img_path in test_images:
    # GT por imagem
    y_true.append(img_true_class(img_path))

    # Predição
    pred_list = detector.predict(source=str(img_path), conf=0.25, iou=0.5, verbose=False)
    res = pred_list[0]
    if res is None or res.boxes is None or len(res.boxes)==0:
        y_pred.append(ID_NO_TUMOR)
        prob = np.zeros(len(CLASSES), dtype=float); prob[ID_NO_TUMOR] = 1.0
        y_prob.append(prob)
        continue

    # pega detecção com MAIOR SCORE
    scores = res.boxes.conf.cpu().numpy()
    cls_ids = res.boxes.cls.cpu().numpy().astype(int)
    k = int(np.argmax(scores))
    cls_hat = int(cls_ids[k]); score_hat = float(scores[k])
    y_pred.append(cls_hat)

    # vetor de probabilidades “soft” (coloca score na classe vencedora)
    prob = np.zeros(len(CLASSES), dtype=float)
    prob[cls_hat] = score_hat
    # opcional: normalizar, mas não é necessário para métricas top-1
    y_prob.append(prob)

y_true = np.array(y_true); y_pred = np.array(y_pred); y_prob = np.vstack(y_prob)

# Métricas de classificação (imagem)
f1m  = f1_score(y_true, y_pred, average="macro")
bal  = balanced_accuracy_score(y_true, y_pred)
try:
    roc  = roc_auc_score(np.eye(len(CLASSES))[y_true], y_prob, multi_class="ovr", average="macro")
except Exception:
    roc  = np.nan

print(f"[YOLO→Classificação] F1 macro={f1m:.4f} | Balanced Acc={bal:.4f} | ROC-AUC={roc:.4f}")
print(classification_report(y_true, y_pred, target_names=CLASSES))

In [ ]:
from sklearn.metrics import (
    f1_score, balanced_accuracy_score, roc_auc_score,
    classification_report, confusion_matrix,
    roc_curve, auc, precision_recall_curve
)
# ROC/PR (macro) por imagem
Y = np.eye(len(CLASSES))[y_true]

# ROC macro
fpr, tpr = {}, {}
for i in range(len(CLASSES)):
    fpr[i], tpr[i], _ = roc_curve(Y[:, i], y_prob[:, i])
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(len(CLASSES))]))
mean_tpr = np.zeros_like(all_fpr)
for i in range(len(CLASSES)):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
mean_tpr /= len(CLASSES)
macro_roc_auc = auc(all_fpr, mean_tpr)

plt.figure(figsize=(6,5))
plt.plot(all_fpr, mean_tpr, lw=2, label=f"Macro ROC (AUC={macro_roc_auc:.3f})")
plt.plot([0,1],[0,1],'--',lw=1)
plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title("ROC (macro)"); plt.legend()
plt.tight_layout(); plt.savefig("roc_macro.png", dpi=150); plt.show()

# PR macro
prec, rec = [], []
grid = np.linspace(0,1,500)
for i in range(len(CLASSES)):
    pi, ri, _ = precision_recall_curve(Y[:, i], y_prob[:, i])
    # garantir eixo x crescente
    prec.append(np.interp(grid, ri[::-1], pi[::-1]))
macro_prec = np.mean(np.vstack(prec), axis=0)
macro_pr_auc = auc(grid, macro_prec)

plt.figure(figsize=(6,5))
plt.plot(grid, macro_prec, lw=2, label=f"Macro PR (AUC={macro_pr_auc:.3f})")
plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title("Precision-Recall (macro)"); plt.legend()
plt.tight_layout(); plt.savefig("pr_macro.png", dpi=150); plt.show()


In [ ]:
def bootstrap_cis(y_true, y_pred, y_prob, n_classes, n_boot=1000, seed=42):
    rng = np.random.default_rng(seed)
    N = len(y_true)
    f1m, bal, roc = [], [], []
    for _ in range(n_boot):
        idx = rng.integers(0, N, N)
        yt = y_true[idx]; yp = y_pred[idx]; pr = y_prob[idx]
        f1m.append(f1_score(yt, yp, average="macro"))
        bal.append(balanced_accuracy_score(yt, yp))
        try:
            roc_auc = roc_auc_score(np.eye(n_classes)[yt], pr, multi_class="ovr", average="macro")
        except Exception:
            roc_auc = np.nan
        roc.append(roc_auc)
    def ci(a):
        a = np.array(a, dtype=float)
        a = a[~np.isnan(a)]
        return np.nanmean(a), np.nanpercentile(a, 2.5), np.nanpercentile(a, 97.5)
    return ci(f1m), ci(bal), ci(roc)

(f1m_mean, f1m_lo, f1m_hi), (bal_mean, bal_lo, bal_hi), (roc_mean, roc_lo, roc_hi) = bootstrap_cis(
    y_true, y_pred, y_prob, n_classes=len(CLASSES), n_boot=1000, seed=123
)

print(f"F1 macro: {f1m_mean:.4f}  (95% CI: {f1m_lo:.4f}–{f1m_hi:.4f})")
print(f"Balanced Acc: {bal_mean:.4f}  (95% CI: {bal_lo:.4f}–{bal_hi:.4f})")
print(f"ROC-AUC macro: {roc_mean:.4f}  (95% CI: {roc_lo:.4f}–{roc_hi:.4f})")

In [ ]:
cm = confusion_matrix(y_true, y_pred, labels=list(range(len(CLASSES))))
plt.figure(figsize=(6,5))
plt.imshow(cm, interpolation='nearest')
plt.title('Confusion Matrix (image-level)')
plt.colorbar()
tick_marks = np.arange(len(CLASSES))
plt.xticks(tick_marks, CLASSES, rotation=45, ha='right')
plt.yticks(tick_marks, CLASSES)
plt.xlabel('Predicted'); plt.ylabel('True'); plt.tight_layout()
plt.savefig("confusion_matrix_image_level.png", dpi=150); plt.show()

# **GERAÇÃO DAS IMAGENS DE TESTE COM O "GROUND TRUTH" E O PREDITO PELO MODELO PARA COMPARAÇÃO** #

In [ ]:
# ==== VISUALIZAÇÃO YOLO: GT vs PRED (TP/FP/FN) ====
#!pip -q install -U ultralytics

import os, csv, math, numpy as np
from pathlib import Path
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

# ===== CONFIG =====
#CLASSES = ["glioma","meningioma","no_tumor","pituitary"] #deixar comentado pois o classes ja foi instanciado no inicio

# aponte para o split desejado (ex.: test)
DATA_ROOT   = Path(OUTPUT_DIR)   # onde tem images/test e labels/test
SPLIT       = "test"                                             # "train" | "val" | "test"
IMAGES_DIR  = DATA_ROOT / "images" / SPLIT
LABELS_DIR  = DATA_ROOT / "labels" / SPLIT

WEIGHTS     = Path("/content/runs/detect/train/weights/best.pt")  # ajuste para o seu best.pt
OUT_DIR     = Path("/content/vis_yolo")
OUT_DIR.mkdir(parents=True, exist_ok=True)

CONF_THRES  = 0.25
IOU_THRES   = 0.5
MAX_IMAGES  = None        # opcional: exibir só N imagens (None = todas)

# ===== UTILS =====
def read_gt_xyxy(txt_path: Path, w: int, h: int):
    """Lê YOLO txt (class cx cy w h) -> listas boxes(xyxy) e labels."""
    boxes, labels = [], []
    if not txt_path.exists():
        return boxes, labels
    for ln in txt_path.read_text().splitlines():
        p = ln.strip().split()
        if len(p) < 5: continue
        c, cx, cy, bw, bh = int(float(p[0])), *map(float, p[1:5])
        x1 = (cx - bw/2.0) * w; y1 = (cy - bh/2.0) * h
        x2 = (cx + bw/2.0) * w; y2 = (cy + bh/2.0) * h
        boxes.append([x1,y1,x2,y2]); labels.append(c)
    return boxes, labels

def iou_xyxy(a, b):
    ax1, ay1, ax2, ay2 = a
    bx1, by1, bx2, by2 = b
    inter_x1 = max(ax1, bx1); inter_y1 = max(ay1, by1)
    inter_x2 = min(ax2, bx2); inter_y2 = min(ay2, by2)
    iw = max(0.0, inter_x2 - inter_x1); ih = max(0.0, inter_y2 - inter_y1)
    inter = iw * ih
    area_a = max(0.0, (ax2-ax1)) * max(0.0, (ay2-ay1))
    area_b = max(0.0, (bx2-bx1)) * max(0.0, (by2-by1))
    union = area_a + area_b - inter + 1e-9
    return inter / union

def match_predictions(gt_boxes, gt_labels, pr_boxes, pr_labels, pr_scores, iou_th=0.5):
    """
    Matching guloso por score.
    Retorna: tps (idxs de preds), fp (idxs de preds), fn (idxs de gts), ious (IoU dos TPs).
    """
    # Converte tudo que vamos indexar/ordenar para numpy (evita -list e garante slicing por idxs)
    pr_scores_np = np.asarray(pr_scores, dtype=float)
    pr_labels_np = np.asarray(pr_labels, dtype=int)
    pr_boxes_np  = np.asarray(pr_boxes, dtype=float)  # shape: (N,4) ou (0,)

    if pr_scores_np.size == 0:
        return [], [], list(range(len(gt_boxes))), []

    # ordenar por score decrescente
    order = np.argsort(-pr_scores_np)

    used_gt = set()
    tps, fp, ious = [], [], []

    for i in order:
        best_j, best_iou = -1, 0.0
        # tenta casar só com GTs da MESMA classe e ainda não usados
        for j, (gbox, glab) in enumerate(zip(gt_boxes, gt_labels)):
            if j in used_gt:
                continue
            if int(pr_labels_np[i]) != int(glab):
                continue
            iou = iou_xyxy(pr_boxes_np[i], gbox)
            if iou > best_iou:
                best_iou, best_j = iou, j

        if best_iou >= iou_th and best_j >= 0:
            used_gt.add(best_j)
            tps.append(int(i))
            ious.append(float(best_iou))
        else:
            fp.append(int(i))

    fn = [j for j in range(len(gt_boxes)) if j not in used_gt]
    return tps, fp, fn, ious

def draw_box(ax, box, color, label=None):
    x1,y1,x2,y2 = box
    ax.add_patch(plt.Rectangle((x1,y1), x2-x1, y2-y1, fill=False, linewidth=2, edgecolor=color))
    if label:
        ax.text(x1, max(0,y1-3), label, fontsize=9, color="white",
                bbox=dict(facecolor=color, edgecolor=color, pad=1.5, alpha=0.7))

def visualize(img_bgr, gt_boxes, gt_labels, pr_boxes, pr_labels, pr_scores, tps, fp, fn, save_prefix):
    img = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    h,w = img.shape[:2]

    # Overlay combinado
    fig, ax = plt.subplots(1,1,figsize=(10,8))
    ax.imshow(img); ax.axis("off")
    # GT (verde)
    for j in range(len(gt_boxes)):
        color = "lime"
        lab = f"GT:{CLASSES[gt_labels[j]]}"
        draw_box(ax, gt_boxes[j], color, lab)
    # TP (azul)
    for i in tps:
        lab = f"TP:{CLASSES[pr_labels[i]]} {pr_scores[i]:.2f}"
        draw_box(ax, pr_boxes[i], "dodgerblue", lab)
    # FP (vermelho)
    for i in fp:
        lab = f"FP:{CLASSES[pr_labels[i]]} {pr_scores[i]:.2f}"
        draw_box(ax, pr_boxes[i], "red", lab)
    # FN (amarelo) -> caixas GT não cobertas
    for j in fn:
        lab = f"FN:{CLASSES[gt_labels[j]]}"
        draw_box(ax, gt_boxes[j], "yellow", lab)
    fig.tight_layout()
    plt.savefig(f"{save_prefix}_overlay.png", dpi=150, bbox_inches="tight")
    plt.close(fig)

    # Lado-a-lado: esquerda GT, direita Pred
    fig, axes = plt.subplots(1,2,figsize=(14,7))
    for k in range(2):
        axes[k].imshow(img); axes[k].axis("off")
    # GT
    for j in range(len(gt_boxes)):
        draw_box(axes[0], gt_boxes[j], "lime", f"GT:{CLASSES[gt_labels[j]]}")
    axes[0].set_title("Ground Truth")
    # Preds (colorindo TP/FP)
    for i in tps:
        draw_box(axes[1], pr_boxes[i], "dodgerblue", f"TP:{CLASSES[pr_labels[i]]} {pr_scores[i]:.2f}")
    for i in fp:
        draw_box(axes[1], pr_boxes[i], "red", f"FP:{CLASSES[pr_labels[i]]} {pr_scores[i]:.2f}")
    axes[1].set_title("Predições YOLO")
    fig.tight_layout()
    plt.savefig(f"{save_prefix}_sidebyside.png", dpi=150, bbox_inches="tight")
    plt.close(fig)

# ===== LOAD MODEL =====
assert IMAGES_DIR.exists() and LABELS_DIR.exists(), "Diretórios de imagens/labels não encontrados."
assert WEIGHTS.exists(), f"Pesos não encontrados: {WEIGHTS}"
#model = YOLO(str(WEIGHTS))
model = detector
# ===== RUN & SAVE =====
image_paths = sorted([p for p in IMAGES_DIR.iterdir() if p.suffix.lower() in {".jpg",".jpeg",".png"}])
if MAX_IMAGES is not None:
    image_paths = image_paths[:MAX_IMAGES]

summary_csv = OUT_DIR / f"summary_{SPLIT}.csv"
with summary_csv.open("w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["image","tp","fp","fn","mean_iou_TP","pred_major(label,score)","gt_major(label)"])

    for img_path in image_paths:
        img = cv2.imread(str(img_path))
        height, width = img.shape[:2]   # ← renomeei aqui

        # GT
        gt_boxes, gt_labels = read_gt_xyxy(LABELS_DIR / (img_path.stem + ".txt"), width, height)

        # Pred
        res = model.predict(source=str(img_path), conf=CONF_THRES, iou=IOU_THRES, verbose=False)[0]
        if res is None or res.boxes is None or len(res.boxes)==0:
            pr_boxes, pr_labels, pr_scores = [], [], []
        else:
            pr_boxes  = res.boxes.xyxy.cpu().numpy()
            pr_labels = res.boxes.cls.cpu().numpy().astype(int)
            pr_scores = res.boxes.conf.cpu().numpy().astype(float)

        # Matching
        tps, fps, fns, ious = match_predictions(gt_boxes, gt_labels, pr_boxes, pr_labels, pr_scores, IOU_THRES)

        # Visual
        save_prefix = str(OUT_DIR / img_path.stem)
        visualize(img, gt_boxes, gt_labels, pr_boxes, pr_labels, pr_scores, tps, fps, fns, save_prefix)

        # resumo por imagem
        if len(pr_scores):
            k = int(np.argmax(pr_scores))
            pred_major = f"{CLASSES[pr_labels[k]]},{pr_scores[k]:.3f}"
        else:
            pred_major = "no_pred,0.000"
        gt_major = CLASSES[gt_labels[np.argmax([b[2]-b[0] for b in gt_boxes])]] if gt_boxes else "no_tumor/empty"
        mean_iou = float(np.mean(ious)) if ious else 0.0

        # usa 'writer' agora
        writer.writerow([img_path.name, len(tps), len(fps), len(fns), f"{mean_iou:.3f}", pred_major, gt_major])


print(f"✅ Visualizações salvas em: {OUT_DIR}")
print(f"CSV resumo: {summary_csv}")


# **VISUALIZAÇÃO DE ALGUMAS DAS IMAGENS GERADAS EM TESTE** #

In [ ]:
import glob
from IPython.display import Image, display

for path in glob.glob("/content/vis_yolo/*_sidebyside.png")[5:20]:  # O index das imagens podem ser limitados para visualização de apenas algumas
    print(path)
    display(Image(filename=path))

# **VISUALIZAÇÃO DE PREDIÇÕES COM FALSO POSITIVO E FALSO NEGATIVO** #

In [ ]:
import pandas as pd
from IPython.display import display, Image
from pathlib import Path

# Ler resumo
df = pd.read_csv(summary_csv)
erros = df[(df['fp'] > 0) | (df['fn'] > 0)].copy()
print(f"Total de imagens com erro: {len(erros)}")
display(erros.head(10))  # mostra a tabela no topo (pode manter)

# Exibir cada linha com sua imagem correspondente
print("\n=== Imagens com erro (FP/FN destacados) ===")

for _, row in erros.iterrows():
    img_name = Path(row["image"]).stem
    img_path = OUT_DIR / f"{img_name}_sidebyside.png"
    if img_path.exists():
        print(f"\n{row['image']} | TP={row['tp']} FP={row['fp']} FN={row['fn']} | mean IoU={row['mean_iou_TP']}")
        print(f"Pred: {row['pred_major(label,score)']} | GT: {row['gt_major(label)']}")
        display(Image(filename=str(img_path)))

# **CONCLUSÕES**

Este estudo experimental teve como objetivo comparar duas arquiteturas de Aprendizado Profundo  para a análise de imagens de ressonância magnética cerebral: (1) uma CNN de Classificação, focada em identificar o tipo de tumor, e (2) um modelo YOLO (Arquitetura 2), focado na deteção e localização do tumor. A CNN de Classificação (Arquitetura 1) alcançou um desempenho robusto na sua tarefa, com uma Acurácia de 95.79% e um F1-Score (macro) de 0.96. Em paralelo, o modelo YOLO (Arquitetura 2) alcançou um mAP (mean Average Precision) de 0.9566 na tarefa de deteção, demonstrando a viabilidade de localizar as lesões em tempo real.

Embora ambas as arquiteturas tenham apresentado resultados promissores, a análise crítica  revela trade-offs significativos.

* CNN (Arquitetura 1): A análise da matriz de confusão e das amostras de erro  mostrou que a CNN teve maior dificuldade em diferenciar as classes Meningioma e Glioma, que são visualmente similares. A sua principal limitação é a necessidade de que a imagem de entrada já esteja "focada" no tumor.


*   YOLO (Arquitetura 2): O YOLO, por outro lado, conseguiu localizar corretamente os tumores mesmo em imagens complexas. Porém, é possivel observar maiores erros em relação a classe Glioma, contudo esses erros podem ser em decorrencia do dataset, ja que é uma arquitetura supervisionada no qual é feito identificação de cada classe manualmente. E visualmente, é possivel detectar a identificação de falsos positivos reportados pela análise, que na verdade o modelo identificou corretamente, porém no dataset não estavam bem identificados.

O custo computacional e de implementação diferiu substancialmente. A CNN de Classificação teve um tempo de treino médio de 5.61 segundos por época  e exigiu um pipeline de dados simples (imagens por pasta). O YOLO, contudo, teria exigido um pré-processamento de dados muito mais complexo, a anotação de bounding boxes, entretanto foi encontrado um dataset já anotado. O custo computacional do treinamento do YOLO foi de 66 segundos por época.

Em conclusão, este estudo demonstrou que não há uma única "melhor" arquitetura, mas sim a arquitetura "certa" para a tarefa. Para uma triagem rápida, a CNN de Classificação é eficiente. No entanto, para um auxílio diagnóstico clínico, que exige localização, o YOLO é superior.

Como próximos passos, propõe-se a implementação de um pipeline híbrido: usar o YOLO (Arquitetura 2) para primeiro detetar e recortar a região de interesse, e então alimentar essa região recortada na CNN de Classificação (Arquitetura 1) para obter uma classificação mais precisa.

